*4*/13/2023:
Experiment
adding recall, precision, accuracy, f1 score, tn, tp, fn, fp





4/1/2023: Change Label to Subject and Patient
FP1 + CZ signals

In [47]:
pip install mne

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [48]:
import os
import numpy as np
import mne
import csv
import matplotlib.pyplot as plt

In [49]:
from sklearn.model_selection import train_test_split
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from keras.models import Sequential
from keras.layers import Dense, SpatialDropout1D
from keras.callbacks import ModelCheckpoint
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
from keras.layers import Dropout


# Set random seed for reproducibility
np.random.seed(42)

In [50]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [51]:
#path='/content/drive/MyDrive/Colab Notebooks/DL/'
path=r'/content/drive/MyDrive/Colab Notebooks/DL/ResampledData/'

length of the signal = observation time (minutes)* 60 (second/minutes) * sampling frequency (1/minutes)

end time = observation time (minutes) * 60 (second/minutes)




---



In [52]:
#observation_time=60 #in minutes
#sampling_frequency=200 #in Hertz or per second
number_of_data=500000
start_time=0
#nd_time=(observation_time*60)
#print(end_time)

We have valid data with total = 46

Create all zeros numpy arrays for FP and CZ signals

In [53]:
number_samples=46
signal_length=number_of_data
fp_cz_signals_o= np.zeros((number_samples, signal_length))

Read only FP1-A2 and CZ-A1:

   (20 subjects)

   (26 subjects) - Patient2 excluded, it does not have CZ signal



Apply bandpass filter:

  l_freq = 1  # low cutoff frequency in Hz

  h_freq = 45  # high cutoff frequency in Hz


Preprocessing: Scale the data to have zero mean and unit variance





In [54]:
file_name=[
                '/subject1_20_raw.fif','/subject2_20_raw.fif',
                '/subject3_20_raw.fif',
                '/subject4_20_raw.fif','/subject5_20_raw.fif',
                '/subject6_20_raw.fif','/subject7_20_raw.fif',
                '/subject8_20_raw.fif',
                '/subject9_20_raw.fif','/subject10_20_raw.fif',
                '/subject11_20_raw.fif','/subject12_20_raw.fif',
                '/subject13_20_raw.fif',
                '/subject14_20_raw.fif','/subject15_20_raw.fif',
                '/subject16_20_raw.fif','/subject17_20_raw.fif',
                '/subject18_20_raw.fif',
                '/subject19_20_raw.fif','/subject20_20_raw.fif',
                '/patient1_20_raw.fif',
                '/patient3_20_raw.fif',
                '/patient4_20_raw.fif','/patient5_20_raw.fif',
                '/patient6_20_raw.fif','/patient7_20_raw.fif',
                '/patient8_20_raw.fif',
                '/patient9_20_raw.fif','/patient10_20_raw.fif',
                '/patient11_20_raw.fif','/patient12_20_raw.fif',
                '/patient13_20_raw.fif',
                '/patient14_20_raw.fif','/patient15_20_raw.fif',
                '/patient16_20_raw.fif','/patient17_20_raw.fif',
                '/patient18_20_raw.fif',
                '/patient19_20_raw.fif','/patient20_20_raw.fif',
                '/patient21_20_raw.fif','/patient22_20_raw.fif',
                '/patient23_20_raw.fif',
                '/patient24_20_raw.fif','/patient25_20_raw.fif',
                '/patient26_20_raw.fif','/patient27_20_raw.fif',
                ]
channel_names = ['FP1-A2', 'CZ-A1']  # Channels are needed for this analysis
l_freq = 1  # low cutoff frequency in Hz
h_freq = 45  # high cutoff frequency in Hz
filter_length = 'auto'  # automatically calculate the filter length

for index in range(len(file_name)):
    #Read only FP1-A2 and CZ-A1
    raw = mne.io.read_raw_fif(path+file_name[index],preload=True,verbose=False)  # include=channel_names (only 'FP1-A2', 'CZ-A1')
    
    #Consider data length of the observation time from the beginning  
    #raw_filtered_cropped = raw.crop(tmin=start_time, tmax=end_time,include_tmax=False,verbose=False)
    raw_data=raw.get_data()
    channels = raw.ch_names
    #print(channels)
    b='FP1-A2'
    c='CZ-A1'
    Index_FP1_A2=channels.index(b)
    Index_CZ_A1=channels.index(c)
    #Read the data into fp_signals and cz-signals    
    fp_cz_signals_o[index] = raw_data[Index_FP1_A2]+raw_data[Index_CZ_A1]
#fp_cz_signals=(fp_cz_signals - np.mean(fp_cz_signals)) / np.std(fp_cz_signals)

#Create label for Sex: Woman and Man
labels=np.array((1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,
                 0,1,1,0,1,0,0,1,1,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,1))  #Sex: 1-woman, 0-man

#labels=np.array((1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,
#                 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0))  # 1-Subject, 0-Patient

# Create label for Age:
#labels2=np.array((1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,0,0,
#                 0,1,1,0,1,0,0,1,1,0,0,0,0,0,1,0,1,1,0,0,0,0,0,1,0,1))  #Age
    

In [55]:
# Reshape input data to include a third dimension of size 1
fp_cz_signals=fp_cz_signals_o.reshape(fp_cz_signals_o.shape[0],fp_cz_signals_o.shape[1],1)

In [56]:
np.random.seed(42)

In [57]:
## Split the data into training and testing datasets for each electrodes (f:FP1-A2 and c:CZ-A1)
#Xfc_train, Xfc_test, yfc_train, yfc_test = train_test_split(fp_cz_signals, labels, test_size=0.2, random_state=42)


In [58]:
Xfc_train, Xfc_test, yfc_train, yfc_test = train_test_split(fp_cz_signals, labels, test_size=0.20, random_state=1)

#Xfc_train, Xfc_val, yfc_train, yfc_val = train_test_split(Xfc_train, yfc_train, test_size=0.25, random_state=1) # 0.25 x 0.8 = 0.2


In [59]:
Xfc_train=(Xfc_train - np.mean(Xfc_train)) / np.std(Xfc_train)
#Xfc_val=(Xfc_val - np.mean(Xfc_train)) / np.std(Xfc_train)
Xfc_test=(Xfc_test - np.mean(Xfc_train)) / np.std(Xfc_train)

In [60]:
# Create a sequential model with a 1D convolutional layer
model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(signal_length, 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [61]:
# Train the model for c:FP1-A2+CZ1-A1
#model.fit(X_train, y_train, epochs=10, batch_size=32)

#model.fit(Xfc_train, yfc_train, epochs=6, batch_size=32)
model.fit(Xfc_train, yfc_train, epochs=4, batch_size=64)

# Evaluate the model
#loss, accuracy = model.evaluate(X_test, y_test)
#loss, accuracy=model.evaluate(Xfc_val)

#print('Test Loss:', loss)
#print('Test Accuracy:', accuracy)

# Make predictions on new data
#y_pred = model.predict(X_test)
#y_pred = model.predict(Xfc_val)
#y_pred=np.ceil(y_pred)
#f1 = f1_score(yfc_val, y_pred)
#print("F1 score: {:.2f}%".format(f1*100))

y_pred=model.predict(Xfc_test)
y_pred=np.ceil(y_pred)
f1 = f1_score(yfc_test, y_pred)

#val_acc_val = accuracy_score(yfc_val, y_pred)
#print(val_acc_val)
print("F1 score: {:.2f}%".format(f1*100))

y_true=yfc_test
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)#, average='macro')
recall = recall_score(y_true, y_pred)#, average='macro')
f1 = f1_score(y_true, y_pred)#, average='macro')

# Print the results
print("Accuracy: {:.2f}%".format(accuracy*100))
print("Precision: {:.2f}%".format(precision*100))
print("Recall: {:.2f}%".format(recall*100))
print("F1 score: {:.2f}%".format(f1*100))



Epoch 1/4
1/1 [==============================] - 21s 21s/step - loss: 0.6967 - accuracy: 0.4722
Epoch 2/4
1/1 [==============================] - 20s 20s/step - loss: 56.8831 - accuracy: 0.5278
Epoch 3/4
1/1 [==============================] - 21s 21s/step - loss: 111.9330 - accuracy: 0.4722
Epoch 4/4
1/1 [==============================] - 1s 1s/step
F1 score: 75.00%
Accuracy: 60.00%
Precision: 66.67%
Recall: 85.71%
F1 score: 75.00%


In [62]:
# Create a sequential model with a 1D convolutional layer
model = Sequential()

model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(signal_length, 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))



# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [63]:
# Train the model for c:FP1-A2+CZ1-A1
#model.fit(X_train, y_train, epochs=10, batch_size=32)

#model.fit(Xfc_train, yfc_train, epochs=6, batch_size=32)
model.fit(Xfc_train, yfc_train, epochs=5, batch_size=64)

# Evaluate the model
#loss, accuracy = model.evaluate(X_test, y_test)
#loss, accuracy=model.evaluate(Xfc_val)

#print('Test Loss:', loss)
#print('Test Accuracy:', accuracy)

# Make predictions on new data
#y_pred = model.predict(X_test)
#y_pred = model.predict(Xfc_val)
#y_pred=np.ceil(y_pred)
#f1 = f1_score(yfc_val, y_pred)
#print("F1 score: {:.2f}%".format(f1*100))

y_pred=model.predict(Xfc_test)
y_pred=np.ceil(y_pred)
f1 = f1_score(yfc_test, y_pred)
print("F1 score: {:.2f}%".format(f1*100))



y_true=yfc_test
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)#, average='macro')
recall = recall_score(y_true, y_pred)#, average='macro')
f1 = f1_score(y_true, y_pred)#, average='macro')

# Print the results
print("Accuracy: {:.2f}%".format(accuracy*100))
print("Precision: {:.2f}%".format(precision*100))
print("Recall: {:.2f}%".format(recall*100))
print("F1 score: {:.2f}%".format(f1*100))



Epoch 1/5
1/1 [==============================] - 15s 15s/step - loss: 0.6739 - accuracy: 0.6111
Epoch 2/5
1/1 [==============================] - 13s 13s/step - loss: 7.2154 - accuracy: 0.8056
Epoch 3/5
1/1 [==============================] - 13s 13s/step - loss: 33.5579 - accuracy: 0.7222
Epoch 4/5
1/1 [==============================] - 13s 13s/step - loss: 68.8089 - accuracy: 0.6389
Epoch 5/5
1/1 [==============================] - 1s 1000ms/step
F1 score: 44.44%
Accuracy: 50.00%
Precision: 100.00%
Recall: 28.57%
F1 score: 44.44%


In [64]:
# Create a sequential model with a 1D convolutional layer
model = Sequential()

model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(signal_length, 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))


model.add(Flatten())
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))



# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



In [65]:
# Train the model for c:FP1-A2+CZ1-A1
#model.fit(X_train, y_train, epochs=10, batch_size=32)

#model.fit(Xfc_train, yfc_train, epochs=6, batch_size=32)
model.fit(Xfc_train, yfc_train, epochs=5, batch_size=64)

# Evaluate the model
#loss, accuracy = model.evaluate(X_test, y_test)
#loss, accuracy=model.evaluate(Xfc_val)

#print('Test Loss:', loss)
#print('Test Accuracy:', accuracy)

# Make predictions on new data
#y_pred = model.predict(X_test)
#y_pred = model.predict(Xfc_val)
#y_pred=np.ceil(y_pred)
#f1 = f1_score(yfc_val, y_pred)
#print("F1 score: {:.2f}%".format(f1*100))

y_pred=model.predict(Xfc_test)
y_pred=np.ceil(y_pred)
f1 = f1_score(yfc_test, y_pred)
print("F1 score: {:.2f}%".format(f1*100))


y_true=yfc_test
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)#, average='macro')
recall = recall_score(y_true, y_pred)#, average='macro')
f1 = f1_score(y_true, y_pred)#, average='macro')

# Print the results
print("Accuracy: {:.2f}%".format(accuracy*100))
print("Precision: {:.2f}%".format(precision*100))
print("Recall: {:.2f}%".format(recall*100))
print("F1 score: {:.2f}%".format(f1*100))

Epoch 1/5
1/1 [==============================] - 17s 17s/step - loss: 0.6939 - accuracy: 0.5833
Epoch 2/5
1/1 [==============================] - 15s 15s/step - loss: 17.8122 - accuracy: 0.5556
Epoch 3/5
1/1 [==============================] - 16s 16s/step - loss: 25.1233 - accuracy: 0.5000
Epoch 4/5
1/1 [==============================] - 16s 16s/step - loss: 0.5719 - accuracy: 0.8889
Epoch 5/5
1/1 [==============================] - 1s 1s/step
F1 score: 82.35%
Accuracy: 70.00%
Precision: 70.00%
Recall: 100.00%
F1 score: 82.35%


In [66]:
# Create a sequential model with a 1D convolutional layer
model = Sequential()

model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(signal_length, 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))



# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



In [67]:
# Train the model for c:FP1-A2+CZ1-A1
#model.fit(X_train, y_train, epochs=10, batch_size=32)

#model.fit(Xfc_train, yfc_train, epochs=6, batch_size=32)
model.fit(Xfc_train, yfc_train, epochs=5, batch_size=64)

# Evaluate the model
#loss, accuracy = model.evaluate(X_test, y_test)
#loss, accuracy=model.evaluate(Xfc_val)

#print('Test Loss:', loss)
#print('Test Accuracy:', accuracy)

# Make predictions on new data
#y_pred = model.predict(X_test)
#y_pred = model.predict(Xfc_val)
#y_pred=np.ceil(y_pred)
#f1 = f1_score(yfc_val, y_pred)
#print("F1 score: {:.2f}%".format(f1*100))

y_pred=model.predict(Xfc_test)
y_pred=np.ceil(y_pred)
f1 = f1_score(yfc_test, y_pred)
print("F1 score: {:.2f}%".format(f1*100))

y_true=yfc_test
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)#, average='macro')
recall = recall_score(y_true, y_pred)#, average='macro')
f1 = f1_score(y_true, y_pred)#, average='macro')

# Print the results
print("Accuracy: {:.2f}%".format(accuracy*100))
print("Precision: {:.2f}%".format(precision*100))
print("Recall: {:.2f}%".format(recall*100))
print("F1 score: {:.2f}%".format(f1*100))

Epoch 1/5
1/1 [==============================] - 18s 18s/step - loss: 0.7017 - accuracy: 0.3889
Epoch 2/5
1/1 [==============================] - 17s 17s/step - loss: 11.5861 - accuracy: 0.5278
Epoch 3/5
1/1 [==============================] - 17s 17s/step - loss: 2.4229 - accuracy: 0.5278
Epoch 4/5
1/1 [==============================] - 17s 17s/step - loss: 0.5723 - accuracy: 0.8056
Epoch 5/5
1/1 [==============================] - 2s 2s/step
F1 score: 82.35%
Accuracy: 70.00%
Precision: 70.00%
Recall: 100.00%
F1 score: 82.35%


In [68]:
# Create a sequential model with a 1D convolutional layer
model = Sequential()

model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(signal_length, 1)))
model.add(MaxPooling1D(pool_size=2))

model.add(Flatten())
model.add(Dense(units=32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=1, activation='sigmoid'))



# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [69]:
# Train the model for c:FP1-A2+CZ1-A1
#model.fit(X_train, y_train, epochs=10, batch_size=32)

#model.fit(Xfc_train, yfc_train, epochs=6, batch_size=32)
model.fit(Xfc_train, yfc_train, epochs=5, batch_size=64)

# Evaluate the model
#loss, accuracy = model.evaluate(X_test, y_test)
#loss, accuracy=model.evaluate(Xfc_val)

#print('Test Loss:', loss)
#print('Test Accuracy:', accuracy)

# Make predictions on new data
#y_pred = model.predict(X_test)
y_pred = model.predict(Xfc_test)
y_pred=np.ceil(y_pred)
f1 = f1_score(yfc_test, y_pred)

print("F1 score: {:.2f}%".format(f1*100))

y_pred=model.predict(Xfc_test)
y_pred=np.ceil(y_pred)
f1 = f1_score(yfc_test, y_pred)
print("F1 score: {:.2f}%".format(f1*100))

Epoch 1/5
1/1 [==============================] - 10s 10s/step - loss: 0.6805 - accuracy: 0.5000
Epoch 2/5
1/1 [==============================] - 9s 9s/step - loss: 68.1944 - accuracy: 0.6667
Epoch 3/5
1/1 [==============================] - 8s 8s/step - loss: 312.7945 - accuracy: 0.4722
Epoch 4/5
1/1 [==============================] - 9s 9s/step - loss: 1.3357 - accuracy: 0.9722
Epoch 5/5
1/1 [==============================] - 1s 827ms/step
F1 score: 82.35%
1/1 [==============================] - 1s 698ms/step
F1 score: 82.35%
